In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import dabest


sys.path.append("../")
from utils.io import read_parameters
from src.plot_trajectories import normalize_speed
import networkx as nx
from griottes import get_cell_properties, generate_delaunay_graph, generate_geometric_graph, plot_2D


cmap = matplotlib.colormaps["seismic_r"]

In [ ]:
# read parameters and key file

parameter_file  = "../base/parameters.yml"
parameters = read_parameters(parameter_file)       

key_file_path = parameters["key_file"]
key_file = pd.read_csv(key_file_path)

print("key file columns:", key_file.columns)
print("key file conditions:", key_file["condition"].unique())

In [ ]:
# create output folder

output_folder = parameters["output_folder"]
print("Output folder:", output_folder)

data_folder = output_folder + "/tracking_data/"

subfolder = "velocity_field"
if not os.path.exists(output_folder + "/" + subfolder):
    os.mkdir(output_folder + "/" + subfolder)

#tracking_data = pd.read_csv(data_folder + "tracking_data.csv") 


In [ ]:
# plot paramters

#condition = 'mosaic_siCTRL_siAlk1'
#condition = 'mosaic_siCTRL_siSMAD4'
# color = 'red'
#treatment = 'siAlk1'

#number_of_tracks_per_condition = 1000

observation_time = parameters["observation_time"]

# limits for min velocity and max velocity in um/h used for the color map
min_vel_lim = -20.0 # um/h
max_vel_lim = 20.0 # um/h

# extend in microns of the coordinate system used for plotting, 
# range is [-max_x, max_x] in x direction and [-max_y, max_y] in y direction
max_x = 600 # um
max_y = 600 # um

obs_time_length_frames = observation_time[1] - observation_time[0]

plot_tracks = False
plot_arrows = True

distance_threshold = parameters.get("distance_threshold", 25) # um, default value if not specified in parameters

In [ ]:
velocity_df = pd.read_csv(output_folder + subfolder + "/velocity_field.csv")

In [ ]:

velocity_corr_df = pd.DataFrame() #columns=velocity_df.columns)

# neighbour_stats_df = pd.DataFrame()

corr_index = 0
# stats_index = 0

for experimentID in velocity_df["EXPERIMENT_ID"].unique():

    condition = velocity_df[velocity_df["EXPERIMENT_ID"] == experimentID]["CONDITION"].unique()[0]
    velocity_df_exp = velocity_df[velocity_df["EXPERIMENT_ID"] == experimentID]

    #velocity_df_exp = velocity_df_exp.rename(columns={"START_X": "x", "START_Y": "y"})
    velocity_df_exp = velocity_df_exp.rename(columns={"END_X": "x", "END_Y": "y"})

    print(velocity_df_exp.columns)

    descriptors = velocity_df_exp.columns

    print("Generating graph...")
    G_delaunay = generate_delaunay_graph(velocity_df_exp[descriptors],
                            descriptors = descriptors,
                            distance=distance_threshold,
                            image_is_2D = True)
        
    print("Graph generated!")

    neighbors_count = pd.DataFrame()
    k = 0

    #red_neighbors_count = {}
    #green_neighbors_count = {}
    green_total = 0
    other_total = 0

    for a, d in G_delaunay.nodes(data=True):
        #print(a)
        print(d)


        neighbors = G_delaunay[a]
        # Count the neighbors with color red
        red_count = sum(1 for neighbor in neighbors if G_delaunay.nodes[neighbor].get('color') == 'red')
        orange_count = sum(1 for neighbor in neighbors if G_delaunay.nodes[neighbor].get('color') == 'orange')
        green_count = sum(1 for neighbor in neighbors if G_delaunay.nodes[neighbor].get('color') == 'green')
        
        vel_x_neighbours = 0
        vel_x_green = 0
        vel_x_other = 0
        vel_y_neighbours = 0
        vel_y_green = 0
        vel_y_other = 0
        
        #for neighbor in neighbors:
        #    orientation_rad = G_delaunay.nodes[neighbor].get("ORIENTATION_RAD")
        #    color = G_delaunay.nodes[neighbor].get("color")
        #    if color == 'green':


        other_count = np.max([orange_count,red_count])


        neighbors_count.at[k, "EXPERIMENT_ID"] = experimentID
        neighbors_count.at[k, "CONDITION"] = condition
        neighbors_count.at[k, "nx_label"] = a
        neighbors_count.at[k, "neighbour_count"] = green_count + other_count
        neighbors_count.at[k, "other_count"] = other_count
        neighbors_count.at[k, "green_count"] = green_count
        neighbors_count.at[k, "color"] = d["color"]
        neighbors_count.at[k, "orientation_deg"] = d["ORIENTATION_DEG"]
        neighbors_count.at[k, "orientation_rad"] = d["ORIENTATION_RAD"]

        if d["color"] == "green":
            green_total += 1
        else:
            other_total += 1   

        k += 1

        # Store the count in the dictionary
        #red_neighbors_count[a] = red_count
        #green_neighbors_count[a] = green_count
    
    green_ratio = np.round(green_total/(green_total + other_total),3)



    neighbors_count = neighbors_count.dropna()

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    sns.histplot(data = neighbors_count,
             x = 'neighbour_count', 
             binwidth = 1,
             ax = ax[0],
             kde=False,
             stat = 'frequency')
    
    neighbour_mean = np.round(neighbors_count["neighbour_count"].mean(),3)
    ax[0].set_title("mean: %s, green ratio: %s" % (neighbour_mean, green_ratio))
    ax[0].set_xlabel("Number of neighbors")


    green_nodes_df = neighbors_count[neighbors_count["color"] == "green"]
    sns.histplot(data = green_nodes_df,
             x = 'green_count', 
             binwidth = 1,
             ax = ax[1],
             kde=False,
             stat = 'frequency')

    green_mean = np.round(green_nodes_df["green_count"].mean(),3)
    ax[1].set_title("mean: %s, expected %s" % (green_mean,  green_ratio*neighbour_mean))
    ax[1].set_xlabel("Number of green neighbors")

    red_nodes_df = neighbors_count[neighbors_count["color"].isin(["red","orange"])]
    sns.histplot(data = red_nodes_df,
             x = 'other_count', 
             binwidth = 1,
             ax = ax[2],
             kde=False,
             stat = 'frequency')
    
    other_mean = np.round(red_nodes_df["other_count"].mean(),3)  
    ax[2].set_title("mean: %s, expected: %s" % (other_mean, (1.0 - green_ratio)*neighbour_mean))
    ax[2].set_xlabel("Number of red neighbors")

    plt.savefig(output_folder + "/%s_neighbour_count.png" % experimentID)

    #stats_index += 1

    neighbors_count["green_ratio"] = green_ratio

    if len(velocity_corr_df.index) == 0:
        velocity_corr_df = neighbors_count.copy()
    else:
        velocity_corr_df = pd.concat([velocity_corr_df, neighbors_count], ignore_index=True)
    
    

    #print(red_neighbors_count)
    #print(green_neighbors_count)

    plot_2D(G_delaunay,
            alpha_line = 0.6,
            scatterpoint_size = 8,
            legend = True,
            legend_fontsize = 14,
            figsize = (7,7))

In [ ]:
velocity_corr_df.to_csv(output_folder + "/velocity_corr.csv", index=False)
velocity_corr_df


In [ ]:
mean_neighbour_count_df = velocity_corr_df.groupby(["EXPERIMENT_ID","CONDITION","color"]).mean().add_suffix('').reset_index()
mean_neighbour_count_df

In [ ]:
plot_other_df = mean_neighbour_count_df[mean_neighbour_count_df['color'].isin(["red","orange"])]

plot_other_df["green_nbr_ratio"] = plot_other_df["green_count"]/(plot_other_df["neighbour_count"])
plot_other_df["clustering"] = plot_other_df["green_ratio"]/plot_other_df["green_nbr_ratio"]

properties_dabest = dabest.load(data = plot_other_df, x="CONDITION", y="clustering",
                          idx=(( 'mosaic_siScr_siScr',  'mosaic_siScr_siCdc42',  'mosaic_siScr_siRac1')))

fig, ax = plt.subplots(1, figsize=(10, 5))

# properties_dabest = dabest.load(data = plot_other_df, x="CONDITION", y="clustering",
#                           idx=(( 'mosaic_siCTRL_siCTRL',  'mosaic_siCTRL_siSMAD4',  'mosaic_siCTRL_siAlk1')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot(ax=ax) 
test.savefig(output_folder + "clustering.pdf")
test.savefig(output_folder + "clustering.png")
#fig, ax = plt.subplots(1, figsize=(15, 5))
#sns.swarmplot(data = plot_other_df,
#            x = 'CONDITION',
#            y = 'clustering',
#            #hue = 'condition',
#            ax = ax)

In [ ]:
plot_df = pd.DataFrame()
k = 0
for experimentID in mean_neighbour_count_df["EXPERIMENT_ID"].unique():
    temp_df = mean_neighbour_count_df[mean_neighbour_count_df["EXPERIMENT_ID"]==experimentID]
    
    red_df = temp_df[temp_df["color"].isin(["red","orange"])]
    red_df["red_green_ratio"] = red_df["other_count"]/red_df["green_count"]
    green_df = temp_df[temp_df["color"] == "green"] 
    green_df["red_green_ratio"] = green_df["other_count"]/green_df["green_count"]

    plot_df.at[k, "EXPERIMENT_ID"] = experimentID
    plot_df.at[k, "CONDITION"] = temp_df["CONDITION"].iloc[0]
    plot_df.at[k, "ratio"] = red_df["red_green_ratio"].iloc[0]/green_df["red_green_ratio"].iloc[0]  
    k += 1
    
display(plot_df)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 5))

properties_dabest = dabest.load(data = plot_df, x="CONDITION", y="ratio",
                          idx=(( 'mosaic_siScr_siScr',  'mosaic_siScr_siCdc42',  'mosaic_siScr_siRac1')))

# properties_dabest = dabest.load(data = plot_df, x="CONDITION", y="ratio",
#                          idx=(( 'mosaic_siCTRL_siCTRL',  'mosaic_siCTRL_siSMAD4',  'mosaic_siCTRL_siAlk1')))


print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot(ax=ax) 
test.savefig(output_folder + "sorting.pdf")
test.savefig(output_folder + "sorting.png")